# Creación de la base de datos de Personajes Mitológicos

Todos los datos para la creación de la base de datos han sido extraídos de [Wikipedia](https://es.wikipedia.org/).

1. Creación de las tablas de mitologías, regiones y continentes.
2. Obtención de la tabla auxiliar con los personajes pertenecientes a categorías específicas.
3. Obtención de la tabla auxiliar con los personajes pertenecientes a varias categorías.
4. Creación de:
    * Tablas de personajes y clases.
    * Tablas con las relaciones entre personajes y mitologías, y personajes y clases.
    * Matriz de adyacencia con las relaciones de los personajes mitológicos basada en los links de Wikipedia.

Se importan las librarías usadas.

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time
from itertools import compress
import re
import unicodedata

In [2]:
# Funcion para obtener el objeto *BeautifulSoup* de la url.

def get_url(url):
    req = requests.get(url)
    if req.status_code == 200:
        document = BeautifulSoup(req.text, "html.parser")
    else:
        document = None
    return document

## 1. Mitologías por región

Se crea una tabla con las mitologías que se scrapearán y la región a la que pertenecen. También se crean tablas para relacionar las regiones con los continentes.

### Obtención de las mitologías

El conjunto de mitologías se obtiene de diferentes formas:

1. [Hijos de la categoría "Mitología por cultura"](https://es.wikipedia.org/wiki/Especial:%C3%81rbolDeCategor%C3%ADas?target=Mitolog%C3%ADa_por_cultura&mode=all)
2. [Descendientes de la categoría "Mitología por continente"](https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente)
3. [Anexo de deidades](https://es.wikipedia.org/wiki/Anexo:Deidades)
4. Mitologías añadidas directamente a la lista

#### Mitologías 1

Se usa *BeautifulSoup*.

In [3]:
url = "https://es.wikipedia.org/wiki/Especial:ÁrbolDeCategorías?target=Mitología_por_cultura&mode=all"

nodos = get_url(url).find("div", class_ = "CategoryTreeChildren").contents

mitologias1 = []
for nodo in nodos[3:]: # los primeros 3 elementos no contienen mitologias
    mitologias1.append(nodo.find("a").text)

In [4]:
mitologias1

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

#### Mitologías 2

Código válido a día 30/08/2021  
Se usa *selenium*.

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Get LATEST driver version for 99.0.4844
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Sergi\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [6]:
url = "https://es.wikipedia.org/wiki/Categoría:Mitología_por_continente"

driver.get(url)

In [7]:
# Funcion para comprobar si el nodo es una mitologia

def es_mitologia(mit_name):
    return(mit_name[:10] == "Mitología " and " en " not in mit_name)

In [8]:
# Funcion para comprobar si la mitologia ya está guardada en la lista

def en_lista(mit_name, lista):
    return(mit_name in lista)

In [9]:
# Funcion para comprobar si el nodo de una mitologia se puede desplegar

def expand_mitologia(mit_name):
    try:
        state = (driver
                 .find_element_by_xpath("//span[@data-ct-title='{}']"
                                        .format(mit_name
                                                .replace(" ", "_")))
                 .get_attribute("title"))
    except:
        return(False)
    return(state == "expandir")

**Tarda mucho en ejecutarse**  
Los nodos de la URL son desplegables, es decir, contienen subcategorías, por lo que en su interior pueden contener más mitologías. La estrategia es ir abriendo los desplegables de mitologías que no se han abierto anteriormente y así obtener todos los nombres de las mitologías. Cuando se abre un desplegable, hay que volver a recorrer todos los nodos debido al formato del HTML y cómo selenium encuentra los nodos.

In [10]:
mitologias2 = []

nodos = (driver
         .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))

while nodos != []:
    
    time.sleep(0.2) # Wikipedia tarda en contestar
    nombre_mitologia = nodos[0].text[:(nodos[0].text.find("(") - 1)]
    
    # si no es mitología o ya está en la lista...
    if (not(es_mitologia(nombre_mitologia)) or en_lista(nombre_mitologia, mitologias2)):
        del nodos[0] # se elimina el nodo
        continue
    
    # si es mitología y no está en la lista...
    mitologias2.append(nombre_mitologia) # se añade a la lista y...
    if expand_mitologia(nombre_mitologia): # se despliega el nodo si es posible
        (driver
         .find_element_by_xpath("//span[@data-ct-title='{}']"
                                .format(nombre_mitologia)
                                .replace(" ", "_"))
         .click())
        nodos = (driver
                 .find_elements_by_xpath("//div[@class='CategoryTreeItem']"))
        
driver.close()

Algunas categorías genéricas contienen " por ", por ejemplo, "Mitologías por región". Estas categorías se eliminan de la lista.

In [11]:
mitologias2 = list(compress(mitologias2, [" por " not in m for m in mitologias2]))

In [12]:
mitologias2

['Mitología de África',
 'Mitología de América',
 'Mitología bereber',
 'Mitología cartaginesa',
 'Mitología egipcia',
 'Mitología guanche',
 'Mitología igbo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología masai',
 'Mitología pigmea',
 'Mitología yoruba',
 'Mitología zulú',
 'Mitología del Caribe',
 'Mitología de América Central',
 'Mitología anishinaabe',
 'Mitología de América del Norte',
 'Mitología afroamericana',
 'Mitología de Costa Rica',
 'Mitología de El Salvador',
 'Mitología talamanqueña',
 'Mitología de Guatemala',
 'Mitología bribri',
 'Mitología mesoamericana',
 'Mitología taína',
 'Mitología canadiense',
 'Mitología mexica',
 'Mitología maya',
 'Mitología tlaxcalteca',
 'Mitología mixteca',
 'Mitología olmeca',
 'Mitología purépecha',
 'Mitología tolteca',
 'Mitología inuit',
 'Mitología circumpolar',
 'Mitología estadounidense',
 'Mitología ainu',
 'Mitología hopi',
 'Mitología pueblo',
 'Mitología mexicana',
 'Mitología de América del Sur',
 'Mitología de As

#### Mitologías 3

Se usa *BeautifulSoup*.

In [13]:
url = "https://es.wikipedia.org/wiki/Anexo:Deidades"

nodos = get_url(url).find("div", id = "toc")
items = [item.text for item in nodos.find_all("span", class_ = "toctext")]

mitologias3 = []
for item in items:
    if item[:10] == "Mitología ":
        mitologias3.append(item)

In [14]:
mitologias3

['Mitología abenaki',
 'Mitología aborigen australiana',
 'Mitología judaica',
 'Mitología del cristianismo',
 'Mitología del catolicismo',
 'Mitología akamba',
 'Mitología akan',
 'Mitología alur',
 'Mitología árabe',
 'Mitología ashanti',
 'Mitología azteca',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología budista',
 'Mitología bushongo',
 'Mitología celta',
 'Mitología china',
 'Mitología chippewa',
 'Mitología coreana',
 'Mitología creek',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología dinka',
 'Mitología efik',
 'Mitología egipcia',
 'Mitología escandinava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología frigia',
 'Mitología griega',
 'Mitología guanche',
 'Mitología guaraní',
 'Mitología haida',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología huron',
 'Mitología igbo',
 'Mitología inca',
 'Mitología inuit',
 'Mitología iroquesa',
 'Mitología isoko',
 'Mitología japonesa',
 'Mitología khoikhoi',
 'Mitología kongo',
 'Mitología kurumba',
 'Mito

#### Mitologías 4

Las anteriores listas no recogen todas las mitologías de Wikipedia. Se añaden 15 mitologías.

In [15]:
mitologias4 = ["Mitología africana", "Mitología americana", "Mitología asiática", "Mitología europea", "Mitología oceánica",
               "Mitología semítica", "Mitología nubia", "Mitología del hinduismo", "Mitología judía", "Mitología vudú",
               "Mitología hawaiana", "Mitología samoana", "Mitología tahitiana", "Mitología turca", "Mitología altaica"]

#### Agregación de las mitologías

In [16]:
lista_mitologias = list(pd.unique(mitologias1 + mitologias2 + mitologias3 + mitologias4))

lista_mitologias

['Mitología afroamericana',
 'Mitología amazónica',
 'Mitología de América del Norte',
 'Mitología árabe',
 'Mitología aborigen australiana',
 'Mitología bereber',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología celta',
 'Mitología cristiana',
 'Mitología egipcia',
 'Mitología eslava',
 'Mitología etrusca',
 'Mitología finlandesa',
 'Mitología germana',
 'Mitología griega',
 'Mitología guanche',
 'Mitología hinduista',
 'Mitología hopi',
 'Mitología igbo',
 'Mitología incaica',
 'Mitología inuit',
 'Mitología irania',
 'Mitología irlandesa',
 'Mitología itálica',
 'Mitología del judaísmo',
 'Mitología kushita',
 'Mitología malgache',
 'Mitología maorí',
 'Mitología mapuche',
 'Mitología masai',
 'Mitología mesoamericana',
 'Mitología mesopotámica',
 'Mitología íbera',
 'Mitología lituana',
 'Mitología maya',
 'Mitología mexica',
 'Mitología pigmea',
 'Mitología pueblo',
 'Mitología romana',
 'Mitología selknam',
 'Mitología taína',
 'Mitología túrquica',
 'Mitología venezol

### Mitologías por región

Se pueden encontrar algunas mitologías en inglés con sus respectivas regiones en el siguiente [link](https://en.wikipedia.org/wiki/List_of_mythologies). Las mitologías restantes sin región se añaden a mano.

#### Obtención en inglés

Se usa *BeautifulSoup*.

Primero se obtienen los elementos de la URL.

In [17]:
url = "https://en.wikipedia.org/wiki/List_of_mythologies"

items = get_url(url).find("div", class_ = "mw-parser-output").contents

items_clean = []
for item in items:
    if not(isinstance(item, str)) and item.name in ["h2", "h3", "h4", "ul"]:
        items_clean.append(item)

del items_clean[0]

In [18]:
# Función para limpiar el texto de la region

def clean_edit(string):
    if string[-6:] == "[edit]":
        return(string[:-6])
    else:
        return(string)

Se recorren los elementos, clasificando las mitologías en su región y continente.

In [19]:
mythologies = pd.DataFrame()
h2 = True
i = 0
while h2:
    if items_clean[i].name == "h2":
        h2 = False
    elif items_clean[i].name == "h3":
        h3 = clean_edit(items_clean[i].text)
        h4 = None
    elif items_clean[i].name == "h4":
        h4 = clean_edit(items_clean[i].text)
    elif items_clean[i].name == "ul":
        myth_split = items_clean[i].text.split("\n")
        for item in myth_split:
            if item[-9:] == "mythology":
                mythologies = mythologies.append(pd.DataFrame([[item, h4 if h4 is not None else h3, h3]]))
    i += 1

mythologies = mythologies.reset_index(drop = True)
mythologies.columns = ["Mythology", "Region", "Continent"]

mythologies

,Mythology,Region,Continent
0,Baluba mythology,Central Africa,Africa
1,Bushongo mythology,Central Africa,Africa
2,Kongo mythology,Central Africa,Africa
3,Lugbara mythology,Central Africa,Africa
4,Mbuti mythology,Central Africa,Africa
...,...,...,...
140,Chilote mythology,South America,Americas
141,Guarani mythology,South America,Americas
142,Inca mythology,South America,Americas
143,Mapuche mythology,South America,Americas


#### Unión en español

Accedemos a las páginas de las mitologías anteriores en inglés y buscamos si existen los artículos en español. En caso afirmativo, obtenemos el nombre de la mitología en español y actualizamos la tabla anterior con las regiones.

In [20]:
mitologias_esp = [None] * len(mythologies)
url_base = "https://en.wikipedia.org/wiki/{}"
for i in range(len(mythologies)):
    url = url_base.format(mythologies["Mythology"][i].replace(" ", "_"))
    page = get_url(url)
    if page is None:
        continue
    es_link = page.find("a", hreflang = "es", class_ = "interlanguage-link-target")
    if es_link is not None:
        pos = es_link.get("title").find(" – Spanish")
        mitologias_esp[i] = es_link.get("title")[:pos]

mythologies["Mitología"] = mitologias_esp

del mythologies["Mythology"]

mythologies = mythologies[mythologies["Mitología"].isin(lista_mitologias)]

In [21]:
mythologies

,Region,Continent,Mitología
3,Central Africa,Africa,Mitología lugbara
11,North Africa,Africa,Mitología egipcia
31,Central Asia,Asia,Mitología túrquica
34,East Asia,Asia,Mitología china
35,East Asia,Asia,Mitología japonesa
36,East Asia,Asia,Mitología coreana
43,South Asia,Asia,Mitología budista
44,South Asia,Asia,Mitología hinduista
58,West Asia / Middle East,Asia,Mitología cristiana
60,West Asia / Middle East,Asia,Mitología judía


Falta añadir la región de las mitologías siguientes. A pesar de que sean muchas, considero que la región es una variable suficientemente importante para añadirla a mano.

In [22]:
set(lista_mitologias) - set(mythologies["Mitología"])

{'Mitología Báltica',
 'Mitología abenaki',
 'Mitología abrahámica',
 'Mitología acadia',
 'Mitología africana',
 'Mitología afroamericana',
 'Mitología aimara',
 'Mitología ainu',
 'Mitología akamba',
 'Mitología akan',
 'Mitología altaica',
 'Mitología alur',
 'Mitología amazónica',
 'Mitología americana',
 'Mitología anatolia',
 'Mitología anglosajona',
 'Mitología anishinaabe',
 'Mitología aragonesa',
 'Mitología argentina',
 'Mitología ashanti',
 'Mitología asiria',
 'Mitología asiática',
 'Mitología asturiana',
 'Mitología azteca',
 'Mitología babilónica',
 'Mitología baganda',
 'Mitología banyarwanda',
 'Mitología bereber',
 'Mitología boliviana',
 'Mitología bribri',
 'Mitología bushongo',
 'Mitología canadiense',
 'Mitología cananea',
 'Mitología cartaginesa',
 'Mitología castellana',
 'Mitología catalana',
 'Mitología chibcha',
 'Mitología chilena',
 'Mitología chippewa',
 'Mitología circumpolar',
 'Mitología colombiana',
 'Mitología dacia',
 'Mitología dahomey',
 'Mitología 

In [23]:
addMyths = [["Eastern Europe", "Europe", "Mitología Báltica"],
            ["North America", "Americas", "Mitología abenaki"],
            ["West Asia / Middle East", "Asia", "Mitología abrahámica"],
            ["West Asia / Middle East", "Asia", "Mitología acadia"],
            ["Africa", "Africa", "Mitología africana"],
            ["African Diasporic", "Africa", "Mitología afroamericana"],
            ["South America", "Americas", "Mitología aimara"],
            ["Northeast Asia", "Asia", "Mitología ainu"],
            ["East Africa", "Africa", "Mitología akamba"],
            ["West Africa", "Africa", "Mitología akan"],
            ["Central Africa", "Africa", "Mitología alur"],
            ["South America", "Americas", "Mitología amazónica"],
            ["Americas", "Americas", "Mitología americana"],
            ["West Asia / Middle East", "Asia", "Mitología anatolia"],
            ["Northern Europe", "Europe", "Mitología anglosajona"],
            ["Central America", "Americas", "Mitología anishinaabe"],
            ["Western Europe", "Europe", "Mitología aragonesa"],
            ["South America", "Americas", "Mitología argentina"],
            ["West Africa", "Africa", "Mitología ashanti"],
            ["West Asia / Middle East", "Asia", "Mitología asiria"],
            ["Asia", "Asia", "Mitología asiática"],
            ["Western Europe", "Europe", "Mitología asturiana"],
            ["Central America", "Americas", "Mitología azteca"],
            ["West Asia / Middle East", "Asia", "Mitología babilónica"],
            ["East Africa", "Africa", "Mitología baganda"],
            ["East Africa", "Africa", "Mitología banyarwanda"],
            ["North Africa", "Africa", "Mitología bereber"],
            ["South America", "Americas", "Mitología boliviana"],
            ["Central America", "Americas", "Mitología bribri"],
            ["Central Africa", "Africa", "Mitología bushongo"],
            ["North America", "Americas", "Mitología canadiense"],
            ["West Asia / Middle East", "Asia", "Mitología cananea"],
            ["North Africa", "Africa", "Mitología cartaginesa"],
            ["Western Europe", "Europe", "Mitología castellana"],
            ["Western Europe", "Europe", "Mitología catalana"],
            ["South America", "Americas", "Mitología chibcha"],
            ["South America", "Americas", "Mitología chilena"],
            ["North America", "Americas", "Mitología chippewa"],
            ["North America", "Americas", "Mitología circumpolar"],
            ["South America", "Americas", "Mitología colombiana"],
            ["Eastern Europe", "Europe", "Mitología dacia"],
            ["West Africa", "Africa", "Mitología dahomey"],
            ["Americas", "Americas", "Mitología de América"],
            ["Central America", "Americas", "Mitología de América Central"],
            ["North America", "Americas", "Mitología de América del Norte"],
            ["South America", "Americas", "Mitología de América del Sur"],
            ["Southern Europe", "Europe", "Mitología de Argólida"],
            ["Asia", "Asia", "Mitología de Asia"],
            ["Northeast Asia", "Asia", "Mitología de Asia Oriental"],
            ["Central America", "Americas", "Mitología de Costa Rica"],
            ["Southern Europe", "Europe", "Mitología de Creta"],
            ["Central America", "Americas", "Mitología de El Salvador"],
            ["Europe", "Europe", "Mitología de Europa"],
            ["Oceania", "Oceania", "Mitología de Fiyi"],
            ["Western Europe", "Europe", "Mitología de Galicia"],
            ["Southern Europe", "Europe", "Mitología de Grecia Central"],
            ["Central America", "Americas", "Mitología de Guatemala"],
            ["Eastern Europe", "Europe", "Mitología de Hungría"],
            ["West Asia / Middle East", "Asia", "Mitología de Levante"],
            ["Oceania", "Oceania", "Mitología de Melanesia"],
            ["Oceania", "Oceania", "Mitología de Micronesia"],
            ["Oceania", "Oceania", "Mitología de Oceanía"],
            ["West Asia / Middle East", "Asia", "Mitología de Oriente Medio"],
            ["Oceania", "Oceania", "Mitología de Polinesia"],
            ["Eastern Europe", "Europe", "Mitología de Rumania"],
            ["Africa", "Africa", "Mitología de África"],
            ["Southern Europe", "Europe", "Mitología de Élide"],
            ["West Asia / Middle East", "Asia", "Mitología del Antiguo Oriente Próximo"],
            ["Central Asia", "Asia", "Mitología del Asia Central"],
            ["South Asia", "Asia", "Mitología del Asia meridional"],
            ["Northeast Asia", "Asia", "Mitología del Asia norte"],
            ["Southeast Asia", "Asia", "Mitología del Asia sureste"],
            ["Central America", "Americas", "Mitología del Caribe"],
            ["Western Europe", "Europe", "Mitología del País Vasco"],
            ["Southern Europe", "Europe", "Mitología del Peloponeso"],
            ["Western Europe", "Europe", "Mitología del catolicismo"],
            ["West Asia / Middle East", "Asia", "Mitología del cristianismo"],
            ["South Asia", "Asia", "Mitología del hinduismo"],
            ["West Asia / Middle East", "Asia", "Mitología del islam"],
            ["West Asia / Middle East", "Asia", "Mitología del judaísmo"],
            ["North Africa", "Africa", "Mitología dinka"],
            ["Southern Europe", "Europe", "Mitología doriana"],
            ["West Africa", "Africa", "Mitología efik"],
            ["Northern Europe", "Europe", "Mitología escandinava"],
            ["Eastern Europe", "Europe", "Mitología eslava"],
            ["Western Europe", "Europe", "Mitología española"],
            ["North America", "Americas", "Mitología estadounidense"],
            ["Europe", "Europe", "Mitología europea"],
            ["Western Europe", "Europe", "Mitología extremeña"],
            ["West Asia / Middle East", "Asia", "Mitología fenicia"],
            ["Southeast Asia", "Asia", "Mitología filipina"],
            ["West Asia / Middle East", "Asia", "Mitología filistea"],
            ["Northern Europe", "Europe", "Mitología finlandesa"],
            ["Western Europe", "Europe", "Mitología francesa"],
            ["West Asia / Middle East", "Asia", "Mitología frigia"],
            ["Western Europe", "Europe", "Mitología gala"],
            ["West Asia / Middle East", "Asia", "Mitología griega de Asia Menor"],
            ["Southern Europe", "Europe", "Mitología griega de Chipre"],
            ["Southern Europe", "Europe", "Mitología griega de Corinto"],
            ["Southern Europe", "Europe", "Mitología griega de Sicilia"],
            ["Southern Europe", "Europe", "Mitología griega de Tesalia"],
            ["Southern Europe", "Europe", "Mitología griega de Tracia"],
            ["Southern Europe", "Europe", "Mitología griega de Épiro"],
            ["West Africa", "Africa", "Mitología guanche"],
            ["North America", "Americas", "Mitología haida"],
            ["West Asia / Middle East", "Asia", "Mitología hattiana"],
            ["Oceania", "Oceania", "Mitología hawaiana"],
            ["South America", "Americas", "Mitología huilliche"],
            ["North America", "Americas", "Mitología huron"],
            ["West Asia / Middle East", "Asia", "Mitología hurrita"],
            ["West Africa", "Africa", "Mitología igbo"],
            ["South America", "Americas", "Mitología inca"],
            ["Eastern Europe", "Europe", "Mitología indoeuropea"],
            ["North America", "Americas", "Mitología iroquesa"],
            ["West Africa", "Africa", "Mitología isoko"],
            ["Southern Europe", "Europe", "Mitología itálica"],
            ["West Asia / Middle East", "Asia", "Mitología judaica"],
            ["Southern Africa", "Africa", "Mitología khoikhoi"],
            ["Central Africa", "Africa", "Mitología kongo"],
            ["West Africa", "Africa", "Mitología kurumba"],
            ["North Africa", "Africa", "Mitología kushita"],
            ["North America", "Americas", "Mitología kwakiutl"],
            ["Southern Europe", "Europe", "Mitología laconia"],
            ["North America", "Americas", "Mitología lakota"],
            ["Northern Europe", "Europe", "Mitología lapona"],
            ["Northern Europe", "Europe", "Mitología letona"],
            ["West Asia / Middle East", "Asia", "Mitología levantina"],
            ["North Africa", "Africa", "Mitología lotuko"],
            ["East Africa", "Africa", "Mitología malgache"],
            ["South America", "Americas", "Mitología mapuche"],
            ["East Africa", "Africa", "Mitología masai"],
            ["Central America", "Americas", "Mitología mesoamericana"],
            ["West Asia / Middle East", "Asia", "Mitología mesopotámica"],
            ["Central America", "Americas", "Mitología mexicana"],
            ["Central America", "Americas", "Mitología mixteca"],
            ["North America", "Americas", "Mitología navajo"],
            ["North Africa", "Africa", "Mitología nubia"],
            ["Oceania", "Oceania", "Mitología oceánica"],
            ["North America", "Americas", "Mitología pawnee"],
            ["South America", "Americas", "Mitología peruana"],
            ["Central Africa", "Africa", "Mitología pigmea"],
            ["Western Europe", "Europe", "Mitología portuguesa"],
            ["North America", "Americas", "Mitología pueblo"],
            ["Central America", "Americas", "Mitología purépecha"],
            ["South America", "Americas", "Mitología quechua"],
            ["Eastern Europe", "Europe", "Mitología rusa"],
            ["Northern Europe", "Europe", "Mitología sajona y anglosajona"],
            ["North America", "Americas", "Mitología salish"],
            ["Southern Europe", "Europe", "Mitología sarda"],
            ["South America", "Americas", "Mitología selknam"],
            ["West Asia / Middle East", "Asia", "Mitología semítica"],
            ["Northeast Asia", "Asia", "Mitología shinto"],
            ["Northeast Asia", "Asia", "Mitología siberiana"],
            ["West Asia / Middle East", "Asia", "Mitología sumeria"],
            ["North America", "Americas", "Mitología séneca"],
            ["Oceania", "Oceania", "Mitología tahitiana"],
            ["Central America", "Americas", "Mitología taína"],
            ["Central America", "Americas", "Mitología tlaxcalteca"],
            ["Central America", "Americas", "Mitología tolteca"],
            ["Southern Europe", "Europe", "Mitología tracia"],
            ["East Africa", "Africa", "Mitología tumbuka"],
            ["Eastern Europe", "Europe", "Mitología ucraniana"],
            ["West Asia / Middle East", "Asia", "Mitología ugaritiche"],
            ["South America", "Americas", "Mitología venezolana"],
            ["African Diasporic", "Africa", "Mitología vudú"],
            ["North America", "Americas", "Mitología winnebago"],
            ["Central America", "Americas", "Mitología wixarrica"],
            ["West Africa", "Africa", "Mitología yoruba"],
            ["Southern Africa", "Africa", "Mitología zulú"],
            ["North America", "Americas", "Mitología zuñi"],
            ["Southern Europe", "Europe", "Mitología ática"],
            ["Western Europe", "Europe", "Mitología íbera"],
            ["Northeast Asia", "Asia", "Mitología turca"],
            ["Northeast Asia", "Asia", "Mitología altaica"]]

In [24]:
mitologias = (mythologies
              .append(pd.DataFrame(addMyths, columns = ["Region", "Continent", "Mitología"]))
              .drop_duplicates())

mitologias.columns = ["Región", "Continente", "Mitología"]

mitologias

,Región,Continente,Mitología
3,Central Africa,Africa,Mitología lugbara
11,North Africa,Africa,Mitología egipcia
31,Central Asia,Asia,Mitología túrquica
34,East Asia,Asia,Mitología china
35,East Asia,Asia,Mitología japonesa
...,...,...,...
169,North America,Americas,Mitología zuñi
170,Southern Europe,Europe,Mitología ática
171,Western Europe,Europe,Mitología íbera
172,Northeast Asia,Asia,Mitología turca


In [25]:
set(mitologias["Región"])

{'Africa',
 'African Diasporic',
 'Americas',
 'Asia',
 'Central Africa',
 'Central America',
 'Central Asia',
 'East Africa',
 'East Asia',
 'Eastern Europe',
 'Europe',
 'North Africa',
 'North America',
 'Northeast Asia',
 'Northern Europe',
 'Oceania',
 'South America',
 'South Asia',
 'Southeast Asia',
 'Southern Africa',
 'Southern Europe',
 'West Africa',
 'West Asia / Middle East',
 'Western Europe'}

Se cambian los nombres de las regiones y los continentes al español.

In [26]:
region_esp = {"Africa" : "África",
              "African Diasporic" : "Diáspora Africana",
              "Americas" : "América",
              #"Asia" : "Asia",
              "Central Africa" : "África Central",
              "Central America" : "América Central",
              "Central Asia" : "Asia Central",
              "East Africa" : "África del Este",
              "East Asia" : "Asia del Este",
              "Eastern Europe" : "Europa del Este",
              "Europe" : "Europa",
              "North Africa" : "África del Norte",
              "North America" : "América del Norte",
              "Northeast Asia" : "Asia del Noreste",
              "Northern Europe" : "Europa del Norte",
              "Oceania" : "Oceanía",
              "South America" : "América del Sur",
              "South Asia" : "Asia del Sur",
              "Southeast Asia" : "Asia del Sureste",
              "Southern Africa" : "África del Sur",
              "Southern Europe" : "Europa del Sur",
              "West Africa" : "África del Oeste",
              "West Asia / Middle East" : "Oriente Medio",
              "Western Europe" : "Europa del Oeste"}

mitologias = mitologias.replace({"Región" : region_esp, "Continente" : region_esp})
mitologias

,Región,Continente,Mitología
3,África Central,África,Mitología lugbara
11,África del Norte,África,Mitología egipcia
31,Asia Central,Asia,Mitología túrquica
34,Asia del Este,Asia,Mitología china
35,Asia del Este,Asia,Mitología japonesa
...,...,...,...
169,América del Norte,América,Mitología zuñi
170,Europa del Sur,Europa,Mitología ática
171,Europa del Oeste,Europa,Mitología íbera
172,Asia del Noreste,Asia,Mitología turca


Se obtiene la tabla de continentes.

In [27]:
conts = pd.DataFrame(mitologias["Continente"].unique(), columns = ["Continente"])
conts

,Continente
0,África
1,Asia
2,Oceanía
3,Europa
4,América


Se obtiene la tabla de regiones con sus respectivos continentes.

In [28]:
regs = mitologias.drop(["Mitología"], axis = 1).drop_duplicates().sort_values(["Continente", "Región"])
regs

,Región,Continente
12,América,América
134,América Central,América
121,América del Norte,América
140,América del Sur,América
20,Asia,Asia
31,Asia Central,Asia
34,Asia del Este,Asia
7,Asia del Noreste,Asia
43,Asia del Sur,Asia
71,Asia del Sureste,Asia


Se obtiene la tabla de mitologías con sus respectivas regiones.

In [29]:
mits = mitologias[["Mitología", "Región"]].sort_values("Mitología").reset_index(drop = True)
mits

,Mitología,Región
0,Mitología Báltica,Europa del Este
1,Mitología abenaki,América del Norte
2,Mitología aborigen australiana,Oceanía
3,Mitología abrahámica,Oriente Medio
4,Mitología acadia,Oriente Medio
...,...,...
211,Mitología zulú,África del Sur
212,Mitología zuñi,América del Norte
213,Mitología árabe,Oriente Medio
214,Mitología ática,Europa del Sur


Problema con mitología repetida con mismo nombre: "Mitolgía Báltica" y "Mitología báltica". Se elimina "Mitología báltica" ya que la categoría es "Categoría:Mitología Báltica".

In [30]:
mits["Mitología"].apply(lambda x: x.lower()).value_counts()

mitología báltica                 2
mitología mesopotámica            1
mitología sajona y anglosajona    1
mitología persa                   1
mitología letona                  1
                                 ..
mitología escocesa                1
mitología argentina               1
mitología bereber                 1
mitología griega de tesalia       1
mitología asiria                  1
Name: Mitología, Length: 215, dtype: int64

In [31]:
mits = mits.drop(mits[mits.Mitología == "Mitología báltica"].index)
mits

,Mitología,Región
0,Mitología Báltica,Europa del Este
1,Mitología abenaki,América del Norte
2,Mitología aborigen australiana,Oceanía
3,Mitología abrahámica,Oriente Medio
4,Mitología acadia,Oriente Medio
...,...,...
211,Mitología zulú,África del Sur
212,Mitología zuñi,América del Norte
213,Mitología árabe,Oriente Medio
214,Mitología ática,Europa del Sur


In [32]:
conts.to_csv("../data/continentes.csv", header = True, index = False)
regs.to_csv("../data/regiones.csv", header = True, index = False)
mits.to_csv("../data/mitologias.csv", header = True, index = False)

## 2. Personajes por categorías específicas

Con la lista de mitologías, se buscan las categorías disponibles en Wikipedia que únicamente contengan personajes mitológicos. Algunas categorías son únicas para cada mitología en particular, así que se añaden a mano.  
De esas categorías, se obtienen todos los personajes.

In [33]:
mitologias = pd.read_csv("../data/mitologias.csv")
mitologias

,Mitología,Región
0,Mitología Báltica,Europa del Este
1,Mitología abenaki,América del Norte
2,Mitología aborigen australiana,Oceanía
3,Mitología abrahámica,Oriente Medio
4,Mitología acadia,Oriente Medio
...,...,...
210,Mitología zulú,África del Sur
211,Mitología zuñi,América del Norte
212,Mitología árabe,Oriente Medio
213,Mitología ática,Europa del Sur


### Obtención de las URLs de las categorías

Primero se obtiene un dataset con todas las URLs de las categorías que scrapearemos más adelante.

Las URLs de las categorías genéricas están formadas por (1) el *tipo de categoría* y (2) la *mitología*.

Ejemplo: `https://es.wikipedia.org/wiki/Categoría:Dioses_griegos` , donde `Dioses` es la categoría y `griegos` es la mitología.

1. Existen diferentes categorías que contienen personajes mitológicos (Deidades, Dioses, Gigantes, Demonios, Dragones ...).  
    Cada mitología tiene sus propias categorías, por lo que he elegido las categorías que he considerado compartidas entre mitologías. Estas categorías son: Personaje, Deidad, Dios, Diosa, Criatura, Dragón, Gigante, Demonio y Héroe.  
    Por otro lado, cada categoría puede tener diversos formatos según la mitología en particular. Por ejemplo, la categoría de "Criatura" puede estar formateada como: `Criaturas`, `Criaturas_mitológicas` y `Criaturas_de_la_mitología`.

In [34]:
categs = {"Personajes_de_la_mitología" : "Personaje", "Deidades" : "Deidad", "Dioses_de_la_mitología" : "Dios",
          "Dioses" : "Dios", "Diosas_de_la_mitología" : "Diosa", "Diosas" : "Diosa", "Criaturas_de_la_mitología" : "Criatura",
          "Criaturas_mitológicas" : "Criatura", "Dragones" : "Dragón", "Gigantes_de_la_mitología" : "Gigante",
          "Demonios_de_la_mitología" : "Demonio", "Demonios": "Demonio", "Héroes_de_la_mitología" : "Héroe"}

2. El formato de la mitología depende tanto del tipo/formato de la categoría como de la propia mitología.  
    Por ejemplo, en la mitología egipcia podemos tener `Dioses_egipcios`, `Diosas_egipcias` y `Deidades_de_la_mitología_egipcia`. En cada caso, el formato de la mitología es diferente.  
    Además, cada mitología tiene su particular transformación cuando cambia de género y número. Los sustantivos del español suelen seguir unas reglas para esta transformación, aunque existen excepciones. Se aplican las 5 reglas más comunes a las mitologías para obtener sus términos clave, por lo que probablemente una de estas claves será correcta.  
    Algunas URLs tienen mayúscula la primera letra de la mitología, por lo que añado estas posibilidades a las claves.

In [35]:
# Funcion para obtener los terminos clave de una mitologia

def clave_mitologia(mit):
    lista = []
    lista.append("_".join(mit.split()[1:])) # elemento 0 en minuscula
    lista.append("_".join(mit.split()[1:-1] + [mit.split()[-1].capitalize()])) # elemento 1 en mayuscula
    for i in range(2):
        lista.append(lista[i] + "s")
        lista.append(lista[i] + "es")
        lista.append(lista[i][0:-1] + "os")
        lista.append(lista[i][0:-1] + "es")
    return(lista)

In [36]:
# Funcion para comprobar si existe la URL de la categoria

def url_buena(url):
    doc = get_url(url)
    if doc != None:
        paginas = doc.find("div", id = "mw-pages")
        if paginas != None:
            return(True)
    return(False)

In [37]:
# Funcion para obtener las posibles URLs de una mitologia

def obtener_urls(mit):
    url_base = "https://es.wikipedia.org/wiki/Categoría:{}_"
    urls = pd.DataFrame()
    keys = clave_mitologia(mit)
    formatos = list(categs.keys())
    for formato in formatos:
        key_found = False
        i = 0
        while (not(key_found) and i < 10):
            url = url_base.format(formato) + keys[i]
            if url_buena(url):
                fila = pd.DataFrame([[url,
                                      mit,
                                      categs[formato]]])
                urls = urls.append(fila)
                key_found = True
            i += 1
    return(urls.drop_duplicates(ignore_index = True))

**Tarda mucho en ejecutarse**  
Se combinan los tipos de categoría y las mitologías y se obtiene una tabla con las categorías existentes en Wikipedia.

In [38]:
urls_categorias = pd.DataFrame()

for mitologia in mitologias["Mitología"]:
    urls_categorias = urls_categorias.append(obtener_urls(mitologia))

urls_categorias.columns = ["URL", "Mitología", "Clase"]

In [39]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología africana,Deidad
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología altaica,Deidad
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología americana,Deidad
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología anatolia,Deidad
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología anglosajona,Dios
...,...,...,...
0,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología vudú,Dios
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología yoruba,Deidad
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología árabe,Deidad
1,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología árabe,Dios


Se añaden directamente las siguientes categorías con formato único:

#### Mitología griega
[Conceptos personificados (deidad)](https://es.wikipedia.org/wiki/Categoría:Conceptos_personificados_de_la_mitología_griega)  
[Grayas (diosa)](https://es.wikipedia.org/wiki/Categoría:Grayas)  
[Horas (diosa)](https://es.wikipedia.org/wiki/Categoría:Personajes_de_la_mitología_griega)  
[Oceánidas (dios)](https://es.wikipedia.org/wiki/Categoría:Oceánidas)  
[Ourea (deidad)](https://es.wikipedia.org/wiki/Categoría:Ourea)  
[Autómatas (criatura)](https://es.wikipedia.org/wiki/Categoría:Autómatas_de_la_mitología_griega)  
[Centauros (criatura)](https://es.wikipedia.org/wiki/Categoría:Centauros)  
[Monstruos (criatura)](https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega)  
[Harpías (criatura)](https://es.wikipedia.org/wiki/Categoría:Harpías)  
[Sátiros (criatura)](https://es.wikipedia.org/wiki/Categoría:Sátiros)  
[Diosas del mar (diosa)](https://es.wikipedia.org/wiki/Categoría:Diosas_del_mar_de_la_mitología_griega)  
[Nereidas (diosa)](https://es.wikipedia.org/wiki/Categoría:Nereidas)  
[Oceánides (diosa)](https://es.wikipedia.org/wiki/Categoría:Oceánides)  
[Erinias (diosa)](https://es.wikipedia.org/wiki/Categoría:Erinias)  
[Musas (diosa)](https://es.wikipedia.org/wiki/Categoría:Musas)  
[Ninfas (diosa)](https://es.wikipedia.org/wiki/Categoría:Ninfas)  
[Dríades (diosa)](https://es.wikipedia.org/wiki/Categoría:Dríades)  
[Náyades (diosa)](https://es.wikipedia.org/wiki/Categoría:Náyades)  
[Pléyades (diosa)](https://es.wikipedia.org/wiki/Categoría:Pléyades)  
[Dioses ctónicos (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_ctónicos_del_panteón_griego)  
[Dioses del mar (dios)](https://es.wikipedia.org/wiki/Categoría:Dioses_del_mar_de_la_mitología_griega)  
[Dioses olímpicos (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos)  
[Dioses olímpicos menores (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores)  
[Dioses primordiales (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega)  
[Titanes (deidad)](https://es.wikipedia.org/wiki/Categoría:Titanes)  
[Dioses del viento (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_griega)  
[Deidades (deidad)](https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega)  
#### Mitología nórdica
[Caballos (criatura)](https://es.wikipedia.org/wiki/Categoría:Caballos_de_la_mitología_nórdica)  
[Lobos (criatura)](https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica)  
[Ásynjur (diosa)](https://es.wikipedia.org/wiki/Categoría:Ásynjur)  
[Nornas (diosa)](https://es.wikipedia.org/wiki/Categoría:Nornas)  
[Æsir (dios)](https://es.wikipedia.org/wiki/Categoría:Æsir)  
[Valquirias (diosa)](https://es.wikipedia.org/wiki/Categoría:Valquirias)  
[Vanir (deidad)](https://es.wikipedia.org/wiki/Categoría:Vanir)  
[Enanos (criatura)](https://es.wikipedia.org/wiki/Categoría:Enanos_en_la_mitología_nórdica)  
[Jotuns (gigante)](https://es.wikipedia.org/wiki/Categoría:Jotuns)  
#### Mitología hinduista
[Avatares (personaje)](https://es.wikipedia.org/wiki/Categoría:Avatares_del_hinduismo)  
[Deidades rigvédicas (deidad)](https://es.wikipedia.org/wiki/Categoría:Deidades_rigvédicas)  
[Razas no humanas (personaje)](https://es.wikipedia.org/wiki/Categoría:Razas_no_humanas_en_la_mitología_hindú)  
[Apsará (diosa)](https://es.wikipedia.org/wiki/Categoría:Apsará)  
[Demonios (demonio)](https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo)  
[Ráksasas (demonio)](https://es.wikipedia.org/wiki/Categoría:Ráksasas)  
#### Mitología japonesa
[Divinidades sintoistas (deidad)](https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas)  
[Yōkai (criatura)](https://es.wikipedia.org/wiki/Categoría:Yōkai)  
[Myō-ō (deidad)](https://es.wikipedia.org/wiki/Categoría:Myō-ō)  
#### Mitología ainu
[Kamui (deidad)](https://es.wikipedia.org/wiki/Categoría:Kamui)  
#### Mitología budista
[Ráksasas (demonio)](https://es.wikipedia.org/wiki/Categoría:Ráksasas)  
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_mitológicas_del_budismo)  
[Deidades femeninas (diosa)](https://es.wikipedia.org/wiki/Categoría:Deidades_femeninas_en_el_budismo)  
[Dakinis (diosa)](https://es.wikipedia.org/wiki/Categoría:Dakinis)  
[Deidades del budismo tibetano (deidad)](https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano)  
[Dharmapalas (deidad)](https://es.wikipedia.org/wiki/Categoría:Dharmapalas)  
[Myō-ō (deidad)](https://es.wikipedia.org/wiki/Categoría:Myō-ō)  
[Devas (deidad)](https://es.wikipedia.org/wiki/Categoría:Devas_(budismo))  
[Seres (personaje)](https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista)  
#### Mitología mexica
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas)  
[Cihuateteo (diosa)](https://es.wikipedia.org/wiki/Categoría:Cihuateteo)  
[Macuiltonaleque (dios)](https://es.wikipedia.org/wiki/Categoría:Macuiltonaleque)  
[Deidades del amor y la fertilidad (deidad)](https://es.wikipedia.org/wiki/Categoría:Deidades_del_amor_y_la_fertilidad_mexica)  
[Centzon Huitznáhuac (deidad)](https://es.wikipedia.org/wiki/Categoría:Centzon_Huitznáhuac)  
[Centzon Totochtin (deidad)](https://es.wikipedia.org/wiki/Categoría:Centzon_Totochtin)  
[Cinteteo (dios)](https://es.wikipedia.org/wiki/Categoría:Cinteteo)  
[Conceptos personificados (deidad)](https://es.wikipedia.org/wiki/Categoría:Conceptos_personificados_de_la_mitología_mexica)  
[Dioses del viento (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica)  
[Dioses primordiales (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_mexica)  
[Divinidades de la muerte (deidad)](https://es.wikipedia.org/wiki/Categoría:Divinidades_de_la_muerte_de_la_mitología_mexica)  
[Tezcatlipocas (deidad)](https://es.wikipedia.org/wiki/Categoría:Tezcatlipocas)  
[Tlaloque (deidad)](https://es.wikipedia.org/wiki/Categoría:Tlaloque)  
[Tonalteteuctin (deidad)](https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin)  
#### Mitología china
[Los ocho inmortales (deidad)](https://es.wikipedia.org/wiki/Categoría:Los_ocho_inmortales)  
[Tres augustos y cinco emperadores (personaje)](https://es.wikipedia.org/wiki/Categoría:Tres_augustos_y_cinco_emperadores)  
#### Mitología celta
[Deidades britanas (deidad)](https://es.wikipedia.org/wiki/Categoría:Deidades_britanas)  
[Fomorianos (deidad)](https://es.wikipedia.org/wiki/Categoría:Fomorianos)  
[Tuatha Dé Danann (deidad)](https://es.wikipedia.org/wiki/Categoría:Tuatha_Dé_Danann)  
#### Mitología irlandesa
[Fomorianos (deidad)](https://es.wikipedia.org/wiki/Categoría:Fomorianos)  
[Tuatha Dé Danann (deidad)](https://es.wikipedia.org/wiki/Categoría:Tuatha_Dé_Danann)  
#### Mitología yoruba
[Orisha (deidad)](https://es.wikipedia.org/wiki/Categoría:Orisha)  
#### Mitología del judaísmo
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_judías)  
[Ángeles (criatura)](https://es.wikipedia.org/wiki/Categoría:Ángeles_en_el_judaísmo)  
#### Mitología persa
[Daevas (demonio)](https://es.wikipedia.org/wiki/Categoría:Daevas)  
[Yazatas (deidad)](https://es.wikipedia.org/wiki/Categoría:Yazatas)  
#### Mitología cántabra
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_del_folclore_de_Cantabria)  
#### Mitología asturiana
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_del_folclore_de_Asturias)  
#### Mitología de América del Norte
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_de_los_pueblos_indígenas_de_América_del_Norte)  
#### Mitología de Oriente Medio
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_de_Oriente_Medio)  
[Dioses (deidad)](https://es.wikipedia.org/wiki/Categoría:Dioses_del_Oriente_Medio)  
[Diosas elamitas (diosa)](https://es.wikipedia.org/wiki/Categoría:Diosas_elamitas)  
[Dioses elamitas (dios)](https://es.wikipedia.org/wiki/Categoría:Dioses_elamitas)  
#### Mitología del islam
[Criaturas (criatura)](https://es.wikipedia.org/wiki/Categoría:Criaturas_mitológicas_islámicas)  
[Ángeles (criatura)](https://es.wikipedia.org/wiki/Categoría:Ángeles_en_el_islam)  

In [40]:
addCategs = [["https://es.wikipedia.org/wiki/Categoría:Conceptos_personificados_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Grayas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Horas_(mitología)","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánidas","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Ourea","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Autómatas_de_la_mitología_griega","Mitología griega","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Centauros","Mitología griega","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Monstruos_de_la_mitología_griega","Mitología griega","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Harpías","Mitología griega","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Sátiros","Mitología griega","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_del_mar_de_la_mitología_griega","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nereidas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Oceánides","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Erinias","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Musas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Ninfas","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dríades","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Náyades","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Pléyades","Mitología griega","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_ctónicos_del_panteón_griego","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_mar_de_la_mitología_griega","Mitología griega","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_olímpicos_menores","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Titanes","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_de_la_mitología_griega","Mitología griega","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Caballos_de_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Lobos_de_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Ásynjur","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Nornas","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Æsir","Mitología nórdica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Valquirias","Mitología nórdica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Vanir","Mitología nórdica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Enanos_en_la_mitología_nórdica","Mitología nórdica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Jotuns","Mitología nórdica","Gigante"],
             ["https://es.wikipedia.org/wiki/Categoría:Avatares_del_hinduismo","Mitología hinduista","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_rigvédicas","Mitología hinduista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Razas_no_humanas_en_la_mitología_hindú","Mitología hinduista","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Apsará","Mitología hinduista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Demonios_en_el_hinduismo","Mitología hinduista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología hinduista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Divinidades_sintoistas","Mitología japonesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Yōkai","Mitología japonesa","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Myō-ō","Mitología japonesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Kamui","Mitología ainu","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Ráksasas","Mitología budista","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mitológicas_del_budismo","Mitología budista","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_femeninas_en_el_budismo","Mitología budista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dakinis","Mitología budista","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_del_budismo_tibetano","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dharmapalas","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Myō-ō","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Devas_(budismo)","Mitología budista","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Seres_de_la_mitología_budista","Mitología budista","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mexicas","Mitología mexica","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Cihuateteo","Mitología mexica","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Macuiltonaleque","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_del_amor_y_la_fertilidad_mexica","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Centzon_Huitznáhuac","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Centzon_Totochtin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Cinteteo","Mitología mexica","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Conceptos_personificados_de_la_mitología_mexica","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_viento_de_la_mitología_mexica","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_primordiales_de_la_mitología_mexica","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Divinidades_de_la_muerte_de_la_mitología_mexica","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tezcatlipocas","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tlaloque","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tonalteteuctin","Mitología mexica","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Los_ocho_inmortales","Mitología china","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tres_augustos_y_cinco_emperadores","Mitología china","Personaje"],
             ["https://es.wikipedia.org/wiki/Categoría:Deidades_britanas","Mitología celta","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Fomorianos","Mitología celta","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tuatha_Dé_Danann","Mitología celta","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Fomorianos","Mitología irlandesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Tuatha_Dé_Danann","Mitología irlandesa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Orisha","Mitología yoruba","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_judías","Mitología del judaísmo","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Ángeles_en_el_judaísmo","Mitología del judaísmo","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Daevas","Mitología persa","Demonio"],
             ["https://es.wikipedia.org/wiki/Categoría:Yazatas","Mitología persa","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_del_folclore_de_Cantabria","Mitología cántabra","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_del_folclore_de_Asturias","Mitología asturiana","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_de_los_pueblos_indígenas_de_América_del_Norte","Mitología de América del Norte","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_legendarias_de_Oriente_Medio","Mitología de Oriente Medio","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_del_Oriente_Medio","Mitología de Oriente Medio","Deidad"],
             ["https://es.wikipedia.org/wiki/Categoría:Diosas_elamitas","Mitología de Oriente Medio","Diosa"],
             ["https://es.wikipedia.org/wiki/Categoría:Dioses_elamitas","Mitología de Oriente Medio","Dios"],
             ["https://es.wikipedia.org/wiki/Categoría:Criaturas_mitológicas_islámicas","Mitología del islam","Criatura"],
             ["https://es.wikipedia.org/wiki/Categoría:Ángeles_en_el_islam","Mitología del islam","Criatura"]]
             

urls_categorias = urls_categorias.append(pd.DataFrame(addCategs, columns = ["URL", "Mitología", "Clase"]))

In [41]:
# Funcion para obtener la URL de la siguiente pagina de personajes en el caso de que exista

def obtener_siguiente(pagina):
    first_link = pagina.find("div", id = "mw-pages").find("a")
    if first_link.text == "página siguiente":
        return "https://es.wikipedia.org" + first_link.get("href")
    elif first_link.text == "página anterior":
        second_link = pagina.find("div", id = "mw-pages").find_all("a")[1]
        if second_link.text == "página siguiente":
            return "https://es.wikipedia.org" + second_link.get("href")

Se añaden las URLs de las categorías con más de una página de personajes.

In [42]:
urls_categorias = urls_categorias.reset_index(drop = True)

for i in range(len(urls_categorias)):
    next_url = obtener_siguiente(get_url(urls_categorias["URL"][i]))
    while next_url is not None:
        new_categoria = pd.DataFrame([[next_url, urls_categorias["Mitología"][i], urls_categorias["Clase"][i]]])
        new_categoria.columns = ["URL", "Mitología", "Clase"]
        urls_categorias = urls_categorias.append(new_categoria)
        next_url = obtener_siguiente(get_url(next_url))

In [43]:
urls_categorias

,URL,Mitología,Clase
0,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología altaica,Deidad
2,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología americana,Deidad
3,https://es.wikipedia.org/wiki/Categoría:Deidad...,Mitología anatolia,Deidad
4,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología anglosajona,Dios
...,...,...,...
180,https://es.wikipedia.org/wiki/Categoría:Dioses...,Mitología de Oriente Medio,Dios
181,https://es.wikipedia.org/wiki/Categoría:Criatu...,Mitología del islam,Criatura
182,https://es.wikipedia.org/wiki/Categoría:Ángele...,Mitología del islam,Criatura
0,https://es.wikipedia.org/w/index.php?title=Cat...,Mitología griega,Personaje


### Obtención de las URLs de los personajes

Ahora se obtiene una tabla con las URLs de los personajes pertenecientes a las categorías extraídas en el subapartado anterior.

In [44]:
# Funcion para limpiar el nombre del personaje

def clean_name(nombre):
    if nombre.find("mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("Mitología") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("(dios)") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("deidad") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    elif nombre.find("personaje") != -1:
        pos = nombre.find("(")
        return nombre[:pos].strip()
    else:
        return(nombre)

Función para obtener los nombres y las URLs de los personajes de la categoría.  
La división alfabética de los personajes puede estar en dos formatos diferentes:  

* Todas las categorias en el mismo nodo
* Cada categoria tiene su nodo

In [45]:
def scrap_categoria(url):
    pagina = get_url(url)
    lista_pers = []
    lista_urls = []
    div_alfabetica = pagina.find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr")
    if div_alfabetica.find("div", class_ = "mw-category") is None: # caso 1
        categorias = div_alfabetica.find_all("h3")
        check_letras = [categoria.text.isalpha() for categoria in categorias]
        personajes = div_alfabetica.find_all("ul")
        for i in range(len(categorias)):
            if check_letras[i]:
                pers = personajes[i]
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})
    else: # caso 2
        categorias = div_alfabetica.find_all("div", class_ = "mw-category-group")
        for categoria in categorias:
            if categoria.find("h3").text.isalpha():
                pers = categoria.find_all("li")
                lista_pers += [clean_name(per.text) for per in pers]
                lista_urls += ["https://es.wikipedia.org" + per.find("a").get("href") for per in pers]
        return pd.DataFrame({"URL" : lista_urls,
                             "Nombre" : lista_pers,
                             "Mitología" : urls_categorias[urls_categorias["URL"] == url].iloc[0,1],
                             "Clase" : urls_categorias[urls_categorias["URL"] == url].iloc[0,2]})

Se obtienen los personajes de las categorías con sus URLs.

In [46]:
urls_personajes1 = pd.DataFrame()

for url in urls_categorias["URL"]:
    urls_personajes1 = urls_personajes1.append(scrap_categoria(url))

In [47]:
urls_personajes1

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Abuk_(mitolog%C3...,Abuk,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Agoyo,Agoyo,Mitología africana,Deidad
2,https://es.wikipedia.org/wiki/Ala_(deidad_igbo),Ala,Mitología africana,Deidad
3,https://es.wikipedia.org/wiki/Chukwu,Chukwu,Mitología africana,Deidad
4,https://es.wikipedia.org/wiki/Dii_Mauri,Dii Mauri,Mitología africana,Deidad
...,...,...,...,...
96,https://es.wikipedia.org/wiki/Yanira,Yanira,Mitología griega,Personaje
97,https://es.wikipedia.org/wiki/Yaso_(rey_de_Orc...,Yaso (rey de Orcómeno),Mitología griega,Personaje
98,https://es.wikipedia.org/wiki/Y%C3%B3bates,Yóbates,Mitología griega,Personaje
99,https://es.wikipedia.org/wiki/Zacinto_(mitolog...,Zacinto,Mitología griega,Personaje


In [48]:
urls_personajes1.to_csv("urls_pers1.csv", index = False)

## 3. Personajes por intersección de categorías

Ahora se obtendrán los artículos que se encuentren en la categoría de alguna mitología y a su vez en una categoría de personaje mitológico. Por ejemplo: se obtiene el personaje "Fenghuang" porque pertenece a las categorías "Mitología china" y "Aves legendarias".

In [49]:
mitologias["Mitología"]

0                   Mitología Báltica
1                   Mitología abenaki
2      Mitología aborigen australiana
3                Mitología abrahámica
4                    Mitología acadia
                    ...              
210                    Mitología zulú
211                    Mitología zuñi
212                   Mitología árabe
213                   Mitología ática
214                   Mitología íbera
Name: Mitología, Length: 215, dtype: object

Se obtienen las páginas de las categorías disponibles de cada mitología.

In [50]:
lista_url_mits = []

for mitologia in mitologias["Mitología"]:
    url = "https://es.wikipedia.org/wiki/Categoría:" + mitologia.replace(" ","_")
    if url_buena(url):
        lista_url_mits.append(url)
        next_url = obtener_siguiente(get_url(url))
        while next_url is not None:
            lista_url_mits.append(next_url)
            next_url = obtener_siguiente(get_url(next_url))

urls_mits = (pd
             .DataFrame(
                 list(mitologias["Mitología"]),
                 index = ["https://es.wikipedia.org/wiki/Categoría:" + m.replace(" ","_") for m in list(mitologias["Mitología"])])
             .loc[lista_url_mits])
urls_mits.columns = ["Mitología"]

urls_mits

,Mitología
https://es.wikipedia.org/wiki/Categoría:Mitología_Báltica,Mitología Báltica
https://es.wikipedia.org/wiki/Categoría:Mitología_aborigen_australiana,Mitología aborigen australiana
https://es.wikipedia.org/wiki/Categoría:Mitología_abrahámica,Mitología abrahámica
https://es.wikipedia.org/wiki/Categoría:Mitología_acadia,Mitología acadia
https://es.wikipedia.org/wiki/Categoría:Mitología_afroamericana,Mitología afroamericana
...,...
https://es.wikipedia.org/wiki/Categoría:Mitología_yoruba,Mitología yoruba
https://es.wikipedia.org/wiki/Categoría:Mitología_zulú,Mitología zulú
https://es.wikipedia.org/wiki/Categoría:Mitología_árabe,Mitología árabe
https://es.wikipedia.org/wiki/Categoría:Mitología_ática,Mitología ática


Se obtienen todos los artículos de las categorías de mitologías.

In [51]:
lista_urls = []
lista_pers = []
lista_mits = []

for url in urls_mits.index:
    nodos = get_url(url).find("div", id = "mw-pages").find("div", class_ = "mw-content-ltr").find_all("a")
    lista_urls = lista_urls + ["https://es.wikipedia.org" + nodo.get("href") for nodo in nodos]
    lista_pers = lista_pers + [clean_name(nodo.text) for nodo in nodos]
    lista_mits = lista_mits + len(nodos) * [urls_mits.loc[url]["Mitología"]]

articulos_mits = pd.DataFrame({"URL" : lista_urls, "Nombre" : lista_pers, "Mitología" : lista_mits})

articulos_mits

,URL,Nombre,Mitología
0,https://es.wikipedia.org/wiki/Perk%C5%ABnas,Perkūnas,Mitología Báltica
1,https://es.wikipedia.org/wiki/Baralku,Baralku,Mitología aborigen australiana
2,https://es.wikipedia.org/wiki/Barnumbir,Barnumbir,Mitología aborigen australiana
3,https://es.wikipedia.org/wiki/Bunyip,Bunyip,Mitología aborigen australiana
4,https://es.wikipedia.org/wiki/El_Sue%C3%B1o_(e...,El Sueño (espiritualidad),Mitología aborigen australiana
...,...,...,...
2573,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera
2574,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera
2575,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera
2576,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera


In [52]:
# Funcion para obtener las categorias de un articulo

def cats_articulo(url):
    art_cats = get_url(url).find("div", id = "mw-normal-catlinks").find("ul").find_all("a")
    return [cats.text for cats in art_cats]

Funciones para comprobar si los artículos pertenecen a categorías de personajes mitológicos.

In [53]:
def check_deidad(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Deidad", "Dios"])])
    return(check)

In [54]:
def check_criatura(lista):
    check = False
    for l in lista:
        check = any([check, any(x in l for x in ["Criatura", "Animales mitológicos", "Arañas mitológicas", "Aves legendarias",
                                                 "Bovinos mitológicos", "Caballos mitológicos", "Caprinos mitológicos",
                                                 "Felinos mitológicos", "Gatos mitológicos", "Leones mitológicos",
                                                 "Peces mitológicos", "Perros mitológicos", "Reptiles mitológicos",
                                                 "Serpientes mitológicas", "Lobos en la mitología",
                                                 "Duendes, gnomos y enanos", "Seres feéricos",
                                                 "Críptido", "Cambiante", "Quimera", "Monstruo"])])
    return(check)

In [55]:
def check_dragon(lista):
    check = False
    for l in lista:
        check = any([check, "Dragones" in l])
    return(check)

In [56]:
def check_gigante(lista):
    check = False
    for l in lista:
        check = any([check, "Gigante" in l])
    return(check)

In [57]:
def check_heroe(lista):
    check = False
    for l in lista:
        check = any([check, "Héroe" in l])
    return(check)

In [58]:
def check_demonio(lista):
    check = False
    for l in lista:
        check = any([check, "Demonio" in l])
    return(check)

In [59]:
def check_personaje(lista):
    check = False
    for l in lista:
        check = any([check, "Personajes mitológicos" in l])
    return(check)

De los artículos anteriores únicamente queremos obtener los personajes mitológicos.

In [60]:
urls_personajes2 = pd.DataFrame()

for i in range(len(articulos_mits)):
    lista_cats = cats_articulo(articulos_mits["URL"][i])
    if check_deidad(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Deidad"]])))
    if check_criatura(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Criatura"]])))
    if check_dragon(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Dragón"]])))
    if check_gigante(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Gigante"]])))
    if check_heroe(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Héroe"]])))
    if check_demonio(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Demonio"]])))
    if check_personaje(lista_cats):
        urls_personajes2 = (urls_personajes2.append(pd.DataFrame([[articulos_mits["URL"][i],
                                                                   articulos_mits["Nombre"][i],
                                                                   articulos_mits["Mitología"][i],
                                                                   "Personaje"]])))

urls_personajes2.columns = ["URL", "Nombre", "Mitología", "Clase"]

In [61]:
urls_personajes2

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Perk%C5%ABnas,Perkūnas,Mitología Báltica,Deidad
0,https://es.wikipedia.org/wiki/Barnumbir,Barnumbir,Mitología aborigen australiana,Deidad
0,https://es.wikipedia.org/wiki/Bunyip,Bunyip,Mitología aborigen australiana,Criatura
0,https://es.wikipedia.org/wiki/Serpiente_Arco_Iris,Serpiente Arco Iris,Mitología aborigen australiana,Criatura
0,https://es.wikipedia.org/wiki/Yowie,Yowie,Mitología aborigen australiana,Criatura
...,...,...,...,...
0,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
0,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


In [62]:
urls_personajes2.to_csv("urls_pers2.csv", index = False)

## 4. Dataset de personajes

En este apartado se obtendrán tanto las tablas con los personajes, como la matriz de adyacencia con los links de los mismos personajes.

Primero se unen las dos tablas auxiliares anteriores.

In [63]:
url_pers = pd.read_csv("urls_pers1.csv").append(pd.read_csv("urls_pers2.csv"))
url_pers

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Abuk_(mitolog%C3...,Abuk,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Agoyo,Agoyo,Mitología africana,Deidad
2,https://es.wikipedia.org/wiki/Ala_(deidad_igbo),Ala,Mitología africana,Deidad
3,https://es.wikipedia.org/wiki/Chukwu,Chukwu,Mitología africana,Deidad
4,https://es.wikipedia.org/wiki/Dii_Mauri,Dii Mauri,Mitología africana,Deidad
...,...,...,...,...
1317,https://es.wikipedia.org/wiki/Aletes_(deidad),Aletes,Mitología íbera,Deidad
1318,https://es.wikipedia.org/wiki/Ataecina,Ataecina,Mitología íbera,Deidad
1319,https://es.wikipedia.org/wiki/Betatun,Betatun,Mitología íbera,Deidad
1320,https://es.wikipedia.org/wiki/Necis,Necis,Mitología íbera,Deidad


Se añaden directamente algunos personajes mitológicos importantes faltantes:
* [Deméter](https://es.wikipedia.org/wiki/Deméter)
* [Poseidón](https://es.wikipedia.org/wiki/Poseidón)
* [Apolo](https://es.wikipedia.org/wiki/Apolo)
* [Dioniso](https://es.wikipedia.org/wiki/Dioniso)
* [Hera](https://es.wikipedia.org/wiki/Hera)
* [Ares](https://es.wikipedia.org/wiki/Ares)
* [Artemisa](https://es.wikipedia.org/wiki/Artemisa)
* [Jasón](https://es.wikipedia.org/wiki/Jasón)
* [Magna Mater](https://es.wikipedia.org/wiki/Magna_Mater)

In [64]:
new_pers = [["https://es.wikipedia.org/wiki/Dem%C3%A9ter","Deméter","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Poseid%C3%B3n","Poseidón","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Apolo","Apolo","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Dioniso","Dioniso","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Hera","Hera","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Ares","Ares","Mitología griega","Dios"],
           ["https://es.wikipedia.org/wiki/Artemisa","Artemisa","Mitología griega","Diosa"],
           ["https://es.wikipedia.org/wiki/Jas%C3%B3n","Jasón","Mitología griega","Héroe"],
           ["https://es.wikipedia.org/wiki/Magna_Mater","Magna Mater","Mitología romana","Diosa"],
           ["https://es.wikipedia.org/wiki/Io_(mitolog%C3%ADa)","Io","Mitología griega","Criatura"],
           ["https://es.wikipedia.org/wiki/Europa_(mitolog%C3%ADa)","Europa","Mitología griega","Personaje"]]

personajes = url_pers.append(pd.DataFrame(new_pers, columns = ["URL", "Nombre", "Mitología", "Clase"])).reset_index(drop = True)

personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Abuk_(mitolog%C3...,Abuk,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Agoyo,Agoyo,Mitología africana,Deidad
2,https://es.wikipedia.org/wiki/Ala_(deidad_igbo),Ala,Mitología africana,Deidad
3,https://es.wikipedia.org/wiki/Chukwu,Chukwu,Mitología africana,Deidad
4,https://es.wikipedia.org/wiki/Dii_Mauri,Dii Mauri,Mitología africana,Deidad
...,...,...,...,...
5247,https://es.wikipedia.org/wiki/Hera,Hera,Mitología griega,Diosa
5248,https://es.wikipedia.org/wiki/Ares,Ares,Mitología griega,Dios
5249,https://es.wikipedia.org/wiki/Artemisa,Artemisa,Mitología griega,Diosa
5250,https://es.wikipedia.org/wiki/Jas%C3%B3n,Jasón,Mitología griega,Héroe


Existen mismas mitologías con nombres diferentes, por ejemplo, "Mitología de América" y "Mitología americana". Así que estandarizamos los nombres.

In [65]:
std_dict = {"Mitología de América" : "Mitología americana",
            "Mitología de Asia" : "Mitología asiática",
            "Mitología de Europa" : "Mitología europea",
            "Mitología de Levante" : "Mitología levantina",
            "Mitología de África" : "Mitología africana",
            "Mitología del País Vasco" : "Mitología vasca",
            "Mitología del hinduismo" : "Mitología hinduista",
            "Mitología griega de Corinto" : "Mitología griega",
            "Mitología griega de Tesalia" : "Mitología griega",
            "Mitología griega de Tracia" : "Mitología griega",
            "Mitología túrquica" : "Mitología turca"}

personajes = personajes.replace({"Mitología" : std_dict})
personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Abuk_(mitolog%C3...,Abuk,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Agoyo,Agoyo,Mitología africana,Deidad
2,https://es.wikipedia.org/wiki/Ala_(deidad_igbo),Ala,Mitología africana,Deidad
3,https://es.wikipedia.org/wiki/Chukwu,Chukwu,Mitología africana,Deidad
4,https://es.wikipedia.org/wiki/Dii_Mauri,Dii Mauri,Mitología africana,Deidad
...,...,...,...,...
5247,https://es.wikipedia.org/wiki/Hera,Hera,Mitología griega,Diosa
5248,https://es.wikipedia.org/wiki/Ares,Ares,Mitología griega,Dios
5249,https://es.wikipedia.org/wiki/Artemisa,Artemisa,Mitología griega,Diosa
5250,https://es.wikipedia.org/wiki/Jas%C3%B3n,Jasón,Mitología griega,Héroe


Se eliminan artículos que considero no son personajes mitológicos.

In [66]:
no_pers = ["Anexo:", "Usuario:", "DC_Comics", "Estatua",
           "Guerra_entre_los_%C3%86sir_y_los_Vanir", "Titanomaquia", # este articulo pertenece a eventos mitologicos
           "Divinidades_hinduistas", "Pante%C3%B3n_maya", "Dioses_egipcios", # estos artículos pertenecen a conjuntos de divinidades
           "Polite%C3%ADsmo_celta"]

In [67]:
personajes = (personajes[[not(any(substring in url for substring in no_pers)) for url in personajes["URL"]]]
              .reset_index(drop = True))

personajes

,URL,Nombre,Mitología,Clase
0,https://es.wikipedia.org/wiki/Abuk_(mitolog%C3...,Abuk,Mitología africana,Deidad
1,https://es.wikipedia.org/wiki/Agoyo,Agoyo,Mitología africana,Deidad
2,https://es.wikipedia.org/wiki/Ala_(deidad_igbo),Ala,Mitología africana,Deidad
3,https://es.wikipedia.org/wiki/Chukwu,Chukwu,Mitología africana,Deidad
4,https://es.wikipedia.org/wiki/Dii_Mauri,Dii Mauri,Mitología africana,Deidad
...,...,...,...,...
5220,https://es.wikipedia.org/wiki/Hera,Hera,Mitología griega,Diosa
5221,https://es.wikipedia.org/wiki/Ares,Ares,Mitología griega,Dios
5222,https://es.wikipedia.org/wiki/Artemisa,Artemisa,Mitología griega,Diosa
5223,https://es.wikipedia.org/wiki/Jas%C3%B3n,Jasón,Mitología griega,Héroe


Se obtiene la tabla de clases. Se elimina la clase "Personaje", ya que contiene las demás clases.

In [79]:
clas = pd.DataFrame(personajes["Clase"].unique(), columns = ["Clase"]).sort_values("Clase").reset_index(drop = True).drop(8)
clas

,Clase
0,Criatura
1,Deidad
2,Demonio
3,Dios
4,Diosa
5,Dragón
6,Gigante
7,Héroe


Se obtiene la tabla de personajes.

In [69]:
pers = personajes[["URL", "Nombre"]].drop_duplicates().sort_values("Nombre").reset_index(drop = True)
pers

,URL,Nombre
0,https://es.wikipedia.org/wiki/Aba_(mitolog%C3%...,Aba
1,https://es.wikipedia.org/wiki/Abaang%C3%BC%C3%AD,Abaangüí
2,https://es.wikipedia.org/wiki/Abante_(centauro),Abante (centauro)
3,https://es.wikipedia.org/wiki/Abante_(hijo_de_...,Abante (hijo de Euridamante)
4,https://es.wikipedia.org/wiki/Abante_(hijo_de_...,Abante (hijo de Melampo)
...,...,...
3644,https://es.wikipedia.org/wiki/%C4%8C%C3%A9nuke,Čénuke
3645,https://es.wikipedia.org/wiki/%C5%8Cgetsuhime,Ōgetsuhime
3646,https://es.wikipedia.org/wiki/%C5%8Ckuninushi,Ōkuninushi
3647,https://es.wikipedia.org/wiki/%C5%8Cmagatoki,Ōmagatoki


Un artículo de Cloris y otro de Ágave son artículos de desambiguación.

In [70]:
pers.groupby(by = ["Nombre"]).size().sort_values(ascending = False)

Nombre
Šakkan                         1
Europa (oceánide)              1
Euritión                       1
Eurite                         1
Eurimedonte (hijo de Minos)    1
                              ..
Nenia                          1
Nemty                          1
Nemain                         1
Nekomusume                     1
Aba                            1
Length: 3649, dtype: int64

### Obtención del texto de los personajes y eliminación de artículos de desambiguación.

Ahora obtendremos el texto de todos los artículos de personajes mitológicos. Aprovechando que se leerán todos los artículos, también se obtienen los links a otros artículos personajes, y se localizan las páginas pertenecientes a la categoría "desambiguación" ya que estos artículos no nos interesan.

In [71]:
# Funcion para obtener todo el texto util del articulo de wikipedia
# Se omiten los apartados de "Enlaces externos", "Referencias", "Fuentes", "Notas", "Vease tambien" y "Bibliografia"

omit = ["enlaces externos", "referencias", "fuentes", "notas", "véase también", "bibliografía"]

def obt_texto(url):
    pag = get_url(url)
    nodos = pag.find("div", class_ = "mw-parser-output").contents
    texto = ""
    h2 = True
    h3 = True
    for i in range(len(nodos)):
        if isinstance(nodos[i], str):
            continue
        if nodos[i].get("class") is not None:
            if ["noprint", "VT rellink", "ext-quick-survey-panel"] in nodos[i].get("class") or "VT ":
                continue
        tipo = nodos[i].name
        if tipo == "h2":
            if len(set([nodos[i].text.lower().find(o) for o in omit])) == 1: # si el string no contiene ninguna de las palabras clave...
                h2 = True
                h3 = True
            else: # si el string contiene alguna de las palabras que se quieren omitir...
                h2 = False
        if tipo == "h3":
            if len(set([nodos[i].text.find(o) for o in omit])) == 1:
                h3 = True
            else:
                h3 = False
        if h2 and h3:
            texto += nodos[i].text
    des = False
    if pag.find("div", class_ = "rellink noprint mw-disambig-page hatnote") is not None:
        des = True
    return({"texto" : texto, "desambiguacion" : des})

In [72]:
# Funcion para obtener los links dirigidos a otros personajes

def row_matriz_ady(url):
    row = [0] * len(pers)
    a_list = get_url(url).find("div", class_ = "mw-parser-output").find_all("a")
    links = ["https://es.wikipedia.org" + str(link.get("href")) for link in a_list]
    for link in links:
        if link in list(pers["URL"]):
            row[pers.index[pers["URL"] == link][0]] = 1
    df = pd.DataFrame([row], index = [url], columns = pers["Nombre"])
    return df

**Tarda mucho en ejecutarse**

In [73]:
textos = []
adj_matrix = pd.DataFrame()
no_desambiguacion = []
for url in pers["URL"]:
    func_texto = obt_texto(url)
    textos.append(func_texto["texto"])
    no_desambiguacion.append(not(func_texto["desambiguacion"]))
    adj_matrix = adj_matrix.append(row_matriz_ady(url))

pers["Texto_RAW"] = textos

Se eliminan los artículos de desambiguación de la tabla de personajes.  

In [74]:
pers = pers[no_desambiguacion][["Nombre", "URL", "Texto_RAW"]]

pers

,Nombre,URL,Texto_RAW
0,Aba,https://es.wikipedia.org/wiki/Aba_(mitolog%C3%...,"En la mitología griega, Aba era una náyade: la..."
1,Abaangüí,https://es.wikipedia.org/wiki/Abaang%C3%BC%C3%AD,Abaangüí o Avá Anguy es el dios de la Luna de...
2,Abante (centauro),https://es.wikipedia.org/wiki/Abante_(centauro),"En la mitología griega, Abante (Ἄβας / Ábas) e..."
3,Abante (hijo de Euridamante),https://es.wikipedia.org/wiki/Abante_(hijo_de_...,"En la mitología griega, Abante fue un héroe tr..."
4,Abante (hijo de Melampo),https://es.wikipedia.org/wiki/Abante_(hijo_de_...,"En la mitología griega, Abante (Ἄβας) fue un h..."
...,...,...,...
3644,Čénuke,https://es.wikipedia.org/wiki/%C4%8C%C3%A9nuke,Čénuke o Cénuke es uno de los más poderosos e ...
3645,Ōgetsuhime,https://es.wikipedia.org/wiki/%C5%8Cgetsuhime,"Ōgetsuhime u Ohogetsuhime (大宜都比売, 大気都比売神, 大宜津比..."
3646,Ōkuninushi,https://es.wikipedia.org/wiki/%C5%8Ckuninushi,"Ōkuninushi (大国主, Ōkuninushi?) es un kami en el..."
3647,Ōmagatoki,https://es.wikipedia.org/wiki/%C5%8Cmagatoki,"Ōmagatoki (逢魔時 or 大禍時, Ōmagatoki?) es un térmi..."


Se obtiene la matriz de relaciones entre personajes.

In [75]:
adj_matrix = adj_matrix.iloc[no_desambiguacion, no_desambiguacion]

adj_matrix

Nombre,Aba,Abaangüí,Abante (centauro),Abante (hijo de Euridamante),Abante (hijo de Melampo),Abante (hijo de Poseidón),Abante (troyano),Abarbarea,Abarbarea de Tiro,Abarta,...,Þjazi,Þrymr,Þrívaldi,Þrúðgelmir,Þrúðr,Čénuke,Ōgetsuhime,Ōkuninushi,Ōmagatoki,Šakkan
https://es.wikipedia.org/wiki/Aba_(mitolog%C3%ADa),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Abaang%C3%BC%C3%AD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Abante_(centauro),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Abante_(hijo_de_Euridamante),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/Abante_(hijo_de_Melampo),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://es.wikipedia.org/wiki/%C4%8C%C3%A9nuke,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C5%8Cgetsuhime,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C5%8Ckuninushi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://es.wikipedia.org/wiki/%C5%8Cmagatoki,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Se obtiene la tabla de relaciones entre personajes y mitologías.

In [76]:
persXmits = personajes.merge(pers)[["Nombre", "Mitología"]].drop_duplicates().sort_values("Nombre").reset_index(drop = True)

persXmits

,Nombre,Mitología
0,Aba,Mitología griega
1,Abaangüí,Mitología guaraní
2,Abante (centauro),Mitología griega
3,Abante (hijo de Euridamante),Mitología griega
4,Abante (hijo de Melampo),Mitología griega
...,...,...
4337,Čénuke,Mitología de América del Sur
4338,Ōgetsuhime,Mitología japonesa
4339,Ōkuninushi,Mitología japonesa
4340,Ōmagatoki,Mitología japonesa


Se obtiene la tabla de relaciones entre personajes y clases.

In [80]:
persXclas = personajes.merge(pers).merge(clas)[["Nombre", "Clase"]].drop_duplicates().sort_values("Nombre").reset_index(drop = True)

persXclas

,Nombre,Clase
0,Aba,Diosa
1,Abaangüí,Deidad
2,Abante (centauro),Criatura
3,Abarbarea,Diosa
4,Abarbarea de Tiro,Diosa
...,...,...
3448,Čénuke,Deidad
3449,Ōgetsuhime,Dios
3450,Ōkuninushi,Deidad
3451,Ōmagatoki,Criatura


In [91]:
clas.to_csv("../data/clases.csv", header = True, index = False, encoding = "utf-8")
pers.to_csv("../data/personajes.csv", header = True, index = False, encoding = "utf-8")
persXmits.to_csv("../data/personajesXmitologias.csv", header = True, index = False, encoding = "utf-8")
persXclas.to_csv("../data/personajesXclases.csv", header = True, index = False, encoding = "utf-8")
adj_matrix.to_csv("../data/adj_matrix.csv", header = True, index = False, encoding = "utf-8")